In [2]:
import numpy as np
from numpy.random import RandomState
from meta import DMMetaManager
meta_man = DMMetaManager(exam_tsv='metadata/exams_metadata.tsv', 
                         img_tsv='metadata/images_crosswalk.tsv', 
                         img_folder='preprocessedData/jpg_org', 
                         img_extension='jpg')

FileNotFoundError: [Errno 2] No such file or directory: 'metadata/images_crosswalk.tsv'

## Generate from flatten image list

In [2]:
img_list, lab_list = meta_man.get_flatten_img_list()
len(img_list)

500

In [3]:
img_list[:5]

['preprocessedData/jpg_org/000135.jpg',
 'preprocessedData/jpg_org/000136.jpg',
 'preprocessedData/jpg_org/000137.jpg',
 'preprocessedData/jpg_org/000138.jpg',
 'preprocessedData/jpg_org/100151.jpg']

In [4]:
lab_list[:5]

[0, 0, 0, 0, 0]

In [5]:
from dm_image import DMImageDataGenerator
img_dat_gen = DMImageDataGenerator(rotation_range=90.)

Using Theano backend.


In [6]:
dat_gen = img_dat_gen.flow_from_img_list(
    img_list, lab_list, batch_size=10, seed=1,
    save_to_dir='./preprocessedData/jpg_batch',
    save_prefix='ran', save_format='jpg')

There are 34 cancer cases and 466 normal cases.


In [7]:
batch_d = dat_gen.next()

In [8]:
print type(batch_d)
print len(batch_d)

<type 'tuple'>
2


In [9]:
print batch_d[0].shape
print batch_d[1].shape

(10, 1, 256, 256)
(10,)


In [10]:
batch_d[1]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

## Generate from exam list

In [11]:
flatten_exam_list = meta_man.get_flatten_exam_list()
print type(flatten_exam_list)
print len(flatten_exam_list)

<type 'list'>
111


In [12]:
flatten_exam_list[5]

(1626, 2, {'L': {'CC':                                        filename
   imageIndex                                     
   2           preprocessedData/jpg_org/111363.jpg,
   'MLO':                                        filename
   imageIndex                                     
   4           preprocessedData/jpg_org/111365.jpg,
   'cancer': 0},
  'R': {'CC':                                        filename
   imageIndex                                     
   1           preprocessedData/jpg_org/111362.jpg,
   'MLO':                                        filename
   imageIndex                                     
   3           preprocessedData/jpg_org/111364.jpg,
   'cancer': 0}})

In [13]:
flatten_exam_list[0][2]['L']['CC'].shape

(1, 1)

In [14]:
def get_nb_img(e, lat, view):
    df = e[2][lat][view]
    if df is None:
        return 0
    else:
        return df.shape[0]
nb_img = [ get_nb_img(e, 'R', 'MLO') for e in flatten_exam_list ]
print max(nb_img)
print np.where(np.array(nb_img) == 3)

3
(array([ 6, 77]),)


In [15]:
img_df = flatten_exam_list[6][2]['R']['MLO']
print type(img_df)
img_df

<class 'pandas.core.frame.DataFrame'>


,filename
imageIndex,
4,preprocessedData/jpg_org/121381.jpg
7,preprocessedData/jpg_org/121390.jpg
8,preprocessedData/jpg_org/121393.jpg


In [16]:
img_df['filename'].sample(1, random_state=RandomState(1)).iloc[0]

'preprocessedData/jpg_org/121381.jpg'

In [17]:
img_df2 = flatten_exam_list[77][2]['R']['MLO']
print type(img_df2)
img_df2

<class 'pandas.core.frame.DataFrame'>


,filename
imageIndex,
4,preprocessedData/jpg_org/578688.jpg
7,preprocessedData/jpg_org/578717.jpg
9,preprocessedData/jpg_org/578732.jpg


In [18]:
img_df2['filename'].sample(1, random_state=RandomState(1)).iloc[0]

'preprocessedData/jpg_org/578688.jpg'

In [19]:
cancer = flatten_exam_list[0][2]['L']['cancer']
print type(cancer)
print cancer

<type 'int'>
0


In [20]:
img_df['filename'].values

array(['preprocessedData/jpg_org/121381.jpg',
       'preprocessedData/jpg_org/121390.jpg',
       'preprocessedData/jpg_org/121393.jpg'], dtype=object)

In [21]:
img_df.loc[img_df.index, 'filename']

imageIndex
4    preprocessedData/jpg_org/121381.jpg
7    preprocessedData/jpg_org/121390.jpg
8    preprocessedData/jpg_org/121393.jpg
Name: filename, dtype: object

In [22]:
type(img_df.index)

pandas.core.index.Int64Index

In [23]:
img_df.loc[img_df.index.tolist(), 'filename']

imageIndex
4    preprocessedData/jpg_org/121381.jpg
7    preprocessedData/jpg_org/121390.jpg
8    preprocessedData/jpg_org/121393.jpg
Name: filename, dtype: object

In [24]:
img_df['filename'].values.tolist()

['preprocessedData/jpg_org/121381.jpg',
 'preprocessedData/jpg_org/121390.jpg',
 'preprocessedData/jpg_org/121393.jpg']

In [25]:
cls_list = [ (e[2]['L']['cancer'], e[2]['R']['cancer']) 
             for e in flatten_exam_list ]
cls_list = np.array(cls_list)
cls_list.shape

(111, 2)

In [26]:
cls_list[:10, :]

array([[0, 0],
       [1, 0],
       [0, 0],
       [0, 0],
       [1, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [1, 0],
       [1, 0]])

In [27]:
flat_classes = cls_list.ravel()
flat_classes = flat_classes[flat_classes != UNIMAGED_INT]
print flat_classes.shape
print np.unique(flat_classes)

(221,)
[0 1]


In [28]:
print 'left cancer:', np.sum(cls_list[:, 0] == 1)
print 'left unimaged:', np.sum(cls_list[:, 0] == UNIMAGED_INT)

left cancer: 10
left unimaged: 1


In [29]:
print 'right cancer:', np.sum(cls_list[:, 1] == 1)
print 'right unimaged:', np.sum(cls_list[:, 1] == UNIMAGED_INT)

right cancer: 4
right unimaged: 0


In [30]:
np.where(cls_list[:, 0] == UNIMAGED_INT)

(array([95]),)

In [31]:
np.sum(cls_list == UNIMAGED_INT)

1

In [32]:
flatten_exam_list[95]

(64073,
 1,
 {'L': {'CC': None, 'MLO': None, 'cancer': -99},
  'R': {'CC':                                        filename
   imageIndex                                     
   1           preprocessedData/jpg_org/608401.jpg,
   'MLO':                                        filename
   imageIndex                                     
   2           preprocessedData/jpg_org/608406.jpg,
   'cancer': 0}})

In [33]:
a = np.zeros((10, 10))
b = np.ones((10, 10))
c = np.stack((a, b), axis=0)
print c.shape

(2, 10, 10)


In [34]:
c = np.stack((a, b), axis=-1)
print c.shape

(10, 10, 2)


In [35]:
dat_gen = img_dat_gen.flow_from_exam_list(
    flatten_exam_list, batch_size=5, seed=1,
    save_to_dir='./preprocessedData/jpg_batch',
    save_prefix='FEX', save_format='jpg')

For left breasts, normal=100, cancer=10, unimaged=1.
For right breasts, normal=107, cancer=4, unimaged=0.


In [36]:
batch_d = dat_gen.next()

In [41]:
batch_x, batch_y = batch_d
print batch_x.shape
print batch_y.shape

(10, 2, 256, 256)
(10,)


In [42]:
batch_y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [43]:
batch_x[0][0]

array([[   0.,    0.,    0., ...,    0.,    0.,    0.],
       [   0.,    0.,    0., ...,    0.,    0.,    0.],
       [   0.,    0.,    0., ...,    0.,    0.,    0.],
       ..., 
       [  74.,  100.,  100., ...,    0.,    0.,    0.],
       [  74.,  100.,   88., ...,    0.,    0.,    0.],
       [ 100.,   88.,   88., ...,    0.,    0.,    0.]])

In [44]:
batch_x[0][1]

array([[   0.,    0.,    0., ...,    0.,    0.,    0.],
       [   0.,    0.,    0., ...,    0.,    0.,    0.],
       [   0.,    0.,    0., ...,    0.,    0.,    0.],
       ..., 
       [ 159.,  167.,  167., ...,    0.,    0.,    0.],
       [ 159.,  167.,  165., ...,    0.,    0.,    0.],
       [ 167.,  165.,  165., ...,    0.,    0.,    0.]])

In [37]:
last_exam_list = meta_man.get_last_exam_list()
print len(last_exam_list)

58


In [38]:
dat_gen2 = img_dat_gen.flow_from_exam_list(
    last_exam_list, batch_size=5, seed=1,
    save_to_dir='./preprocessedData/jpg_batch',
    save_prefix='LEX', save_format='jpg')

For left breasts, normal=49, cancer=8, unimaged=1.
For right breasts, normal=55, cancer=3, unimaged=0.


In [39]:
batch_d2 = dat_gen2.next()